# SQLSensei Source Code

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk, interleave_datasets
from transformers import AutoTokenizer, TrainingArguments, GenerationConfig, Trainer, AutoModelForSeq2SeqLM
import evaluate
import pandas as pd
import numpy as np
import torch
import time
import warnings
from google.colab import userdata
import os
import random

In [ ]:
# All dependencies automatmatically update
# Future warnings can be safely ignored
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Import hugging face token (Running on Colab)
userdata.get('huggingface')

# Manually input hugging face token (Running Locally)
hugging_face_token = 'token'

# This notebook also requires a wandb API token to be entered when prompted

# Choose the model
model='t5-small'

# Load the model's tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Load the pre-trained model
base_model = AutoModelForSeq2SeqLM.from_pretrained(model, torch_dtype=torch.bfloat16)

# Load Datasets from Hugging Face

Each dataset is split into training and testing, and irrelevant columns are removed from each set. Then, they are merged into a single dataset.

In [ ]:
# Load each dataset
text_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]')
text_test = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]')
text_validation = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]')

context_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]')
context_test = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]')
context_validation = load_dataset("b-mc2/sql-create-context", split='train[-10%:]')

know_train = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
know_test = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
know_validation = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')

# Preprocess text-to-sql dataset
text_train = text_train.remove_columns(['source', 'text'])
text_train = text_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})
text_test = text_test.remove_columns(['source', 'text'])
text_test = text_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})
text_validation = text_validation.remove_columns(['source', 'text'])
text_validation = text_validation.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

# Merge dataset
merged_dataset = DatasetDict({ 'train': interleave_datasets([context_train, text_train, know_train]),
                            'test': interleave_datasets([context_test, text_test, know_test]),
                            'validation': interleave_datasets([context_validation, text_validation, know_validation])})

# Save merged dataset
merged_dataset.save_to_disk("merged_dataset")

# Load merged dataset
merged_dataset = load_from_disk("merged_dataset")

# Data Preprocessing

Here, the data are cleaned and formatted with an NLP function. Then, the data are tokenized and loaded.

In [ ]:
# Define NLP function
def nlp_fn(sql_input):
    """This function takes a SQL input and performs NLP on it.

    Arguments
    ---------
    sql_input: dict; dictionary containing the SQL context and question

    Returns
    -------
    SQL input, tokenized and ready for model usage
    """

    # Pack context and question together
    data_zip = zip(sql_input['context'], sql_input['question'])

    # Generate appropriate prompt for each query
    prompt = ["Tables:\n" + context + "\n\nQuestion:\n" + question + "\n\nAnswer:\n" for context, question in data_zip]

    # Tokenize the IDs
    sql_input['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids

    # Tokenize the labels
    sql_input['labels'] = tokenizer(sql_input['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return sql_input

In [ ]:
# Apply the NLP function on the whole dataset in batches for each split
tokenized_datasets = merged_dataset.map(nlp_fn, batched=True)

# Remove non-tokenized columns
tokenized_datasets = tokenized_datasets.remove_columns(['question', 'context', 'answer'])

In [ ]:
# Save tokenized datasets
tokenized_datasets.save_to_disk("tokenized_datasets")

# Load tokenized datasets
tokenized_datasets = load_from_disk("tokenized_datasets")

In [ ]:
# Verify dataset dimensions
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Testing: {tokenized_datasets['test'].shape}")

# Fine-Tuning

### Hyperparameters:
Learning Rate: $5$ x $10^{-3}$

Warmup Steps (Stabilization): $100$

Epochs: $2$

Training/Evaluation Batch Size: $16$

Weight Decay (Regularization): $0.01$

In [ ]:
# Check if a deployable copy of the model already exists
if os.path.exists(model_path):
    # Load the model if it exists
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("SQLSensei", token=hugging_face_token)
    train = False
else:
    # Prepare the model for training otherwise
    train = True
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("SQLSensei", torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
# Track training time
%%time

if train:

    # Training arguments tuned to NVIDIA GeForce GTX 1060 limitations
    # Sets unique training directory on Hugging Face dashboard
    training_args = TrainingArguments(
        output_dir=f'./sql-training-{str(int(time.time()))}',
        learning_rate=5e-3,
        warmup_steps=100,
        num_train_epochs=2,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        logging_steps=35,
        eval_strategy='epoch'
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
    )

    # Train the model
    trainer.train()

    # Save the fine tuned model
    finetuned_model.save_pretrained("SQLSensei")

In [ ]:
# Load the fine tuned model
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("SQLSensei", token=hugging_face_token)

# Testing SQLSensei

Uses zero shot inference, SQLSensei can now be tested on some specific examples of the testing set.

In [ ]:
# Test on the 5th query of the dataset
query_index = 5

# Set the prompt format to input to the model
prompt = f"""

You are a powerful SQL AI capable of generating accurate SQL queries.

Generate an answer query based on the given information.

Tables:
{dataset['test'][query_index]['context']}

Question:
{dataset['test'][query_index]['question']}

Answer:
"""

# Tokenize the prompt and set as input
inputs = tokenizer(prompt, return_tensors='pt')

# Run the model on the input and store the output
output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

In [ ]:
# Print comparison results
print(f"Input Prompt:\n{prompt}")
print()
print(f"Target Answer:\n{dataset['test'][query_index]['answer']}\n")
print()
print(f"SQLSensei's Answer:\n{output}")

# Evaluate using ROUGE Metric
Now that SQLSensei has proven its ability to generate at least one correct query, it can be evaluated on a small subset of the data to obtain the ROGUE metric for accuracy.

In [ ]:
# Only 20 references are used, for time convienience
# Select a subset of 20 random queries
start = random.randint(0, 100)

end = start + 20

questions = dataset['test'][start:end]['question']
context = dataset['test'][start:end]['context']
target_answers = dataset['test'][start:end]['answer']

# Initiate empty lists to store outputs
base_model_output = []
finetuned_model_output = []

# Run on every query in the subset
for i, question in enumerate(questions):

    # Define prompt for each query and context
    prompt = f"""Tables:
    {context[i]}

    Question:
    {question}

    Answer:
    """

    # Tokenize the input
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # Extract the labeled target output
    human_baseline_text_output = target_answers[i]

    # Extract tokenized base model and SQLSensei output
    base_model_outputs = base_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))

    # Extract base model and SQLSensei output as readable text
    base_model_text_output = tokenizer.decode(base_model_outputs[0], skip_special_tokens=True)
    finetuned_model_text_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)

    # Append base model and SQLSensei readable text output to list
    base_model_output.append(base_model_text_output)
    finetuned_model_output.append(finetuned_model_text_output)

# Zip into tuple, convert to list
zipped_outputs = list(zip(target_answers, base_model_output, finetuned_model_output))

# Convert list to pandas data frame (can be used for visualization/storage)
output_df = pd.DataFrame(zipped_outputs, columns = ['target_answers', 'base_model_output', 'finetuned_model_output'])

In [ ]:
# Compute ROGUE accuracy score for both models
rouge = evaluate.load('rouge')

base_model_results = rouge.compute(
    predictions=base_model_output,
    references=target_answers[0:len(base_model_output)],
    use_aggregator=True,
    use_stemmer=True,
)
print('Base T5 Model Metrics:')
print(base_model_results)

finetuned_model_results = rouge.compute(
    predictions=finetuned_model_output,
    references=target_answers[0:len(finetuned_model_output)],
    use_aggregator=True,
    use_stemmer=True,
)

print('SQLSensei Model Metrics:')
print(finetuned_model_results)

SQLSensei Testing Accuracy: 90%